In [ ]:
import os
import json
import glob
import random
import shutil 
from sklearn.model_selection import train_test_split 

DATASET_DIR = r"D:\sem 5\DL\Project\dataset2\Trucks" 

output_base_dir = r"D:\sem 5\DL\Project\dataset2\Trucks_YOLO"
train_dir = os.path.join(output_base_dir, "train")
val_dir = os.path.join(output_base_dir, "val")

# Define subdirectories for images and labels
train_images_dir = os.path.join(train_dir, "images")
train_labels_dir = os.path.join(train_dir, "labels")
val_images_dir = os.path.join(val_dir, "images")
val_labels_dir = os.path.join(val_dir, "labels")


def load_axle_data(dataset_path):
    """
    Loads all .jpg/.JPG/.jpeg/.JPEG/.png/.PNG and .json pairs from the dataset folder.
    """

    image_extensions = ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG', '*.png', '*.PNG']
    image_paths = []
    for ext in image_extensions:
        image_paths.extend(glob.glob(os.path.join(dataset_path, ext)))
    print(f"Found {len(image_paths)} images. Now matching with JSON...")
    dataset = []

    loaded_count = 0
    missing_json_count = 0
    error_loading_count = 0

    for img_path in image_paths:
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        json_path = os.path.join(dataset_path, base_name + ".json")

        if not os.path.exists(json_path):
            missing_json_count += 1
            continue

        try:
            with open(json_path, 'r', encoding='utf-8') as f: 
                data = json.load(f)
        except Exception as e:
            print(f"Error loading {json_path}: {e}")
            error_loading_count += 1
            continue

        boxes = []
        for ann in data.get("shapes", []):
            label = ann.get("label", "").strip().lower()
            if label != "axle":
                continue

            shape_type = ann.get("shape_type")
            points = ann.get("points", [])

            if shape_type == "rectangle" and len(points) == 2:
                x1, y1 = points[0]
                x2, y2 = points[1]
                xmin = min(x1, x2)
                ymin = min(y1, y2)
                xmax = max(x1, x2)
                ymax = max(y1, y2)
                boxes.append([xmin, ymin, xmax, ymax])

            elif shape_type == "circle" and len(points) == 2:
                x1, y1 = points[0]
                x2, y2 = points[1]
                xmin = min(x1, x2)
                ymin = min(y1, y2)
                xmax = max(x1, x2)
                ymax = max(y1, y2)
                boxes.append([xmin, ymin, xmax, ymax])

            else:
                print(f"Warning: Unsupported or malformed Axle annotation in {json_path}: "
                      f"Shape={shape_type}, Points={points}")


        # Get image dimensions from JSON
        img_width = data.get("imageWidth")
        img_height = data.get("imageHeight")

        # Basic check if dimensions were found
        if not isinstance(img_width, (int, float)) or not isinstance(img_height, (int, float)):
             print(f"Warning: Invalid or missing image dimensions in {json_path}. Width: {img_width}, Height: {img_height}. Skipping.")
             error_loading_count += 1
             continue


        dataset.append({
            "image_path": img_path,
            "json_path": json_path,
            "width": float(img_width), 
            "height": float(img_height),
            "axles": boxes
        })
        loaded_count += 1

    print(f"Finished loading. Successfully processed: {loaded_count}, Missing JSON: {missing_json_count}, Errors/Skipped: {error_loading_count}")
    return dataset

def convert_to_yolo(data_list, img_output_dir, lbl_output_dir, class_index=0):
    """ Converts data points to YOLO format and copies images. """
    count = 0
    skipped_no_axles = 0
    skipped_dimension_error = 0
    skipped_write_error = 0
    skipped_copy_error = 0

    for item in data_list:
        img_path = item['image_path']
        img_width = item['width']
        img_height = item['height']
        axles = item['axles'] 

        # (double check) if image dimensions are invalid 
        if not img_width or not img_height or img_width <= 0 or img_height <= 0:
            print(f"Warning: Skipping {os.path.basename(img_path)} due to invalid dimensions W:{img_width} H:{img_height}.")
            skipped_dimension_error += 1
            continue


        base_filename = os.path.basename(img_path)
        label_filename = os.path.splitext(base_filename)[0] + ".txt"
        label_filepath = os.path.join(lbl_output_dir, label_filename)

        yolo_lines = []
        valid_box_found = False
        for box in axles:
            x1, y1, x2, y2 = box

            if x1 >= x2 or y1 >= y2:
                print(f"Warning: Invalid box coordinates {box} in {os.path.basename(item['json_path'])}. Skipping box.")
                continue

            # Ensure coordinates are within image bounds (clamp if slightly outside)
            x1 = max(0.0, x1)
            y1 = max(0.0, y1)
            x2 = min(img_width, x2)
            y2 = min(img_height, y2)

            # Recalculate dimensions after clamping
            box_width = x2 - x1
            box_height = y2 - y1
            center_x = x1 + (box_width / 2)
            center_y = y1 + (box_height / 2)

            # Check for zero width/height after clamping
            if box_width <= 0 or box_height <= 0:
                 print(f"Warning: Box {box} in {os.path.basename(item['json_path'])} has zero width/height after clamping. Skipping box.")
                 continue


            # Normalize coordinates
            norm_center_x = center_x / img_width
            norm_center_y = center_y / img_height
            norm_width = box_width / img_width
            norm_height = box_height / img_height

            # check normalized values 
            if not (0 <= norm_center_x <= 1 and 0 <= norm_center_y <= 1 and 0 < norm_width <= 1 and 0 < norm_height <= 1):
                 print(f"Warning: Invalid normalized values for box {box} in {os.path.basename(item['json_path'])}. Skipping box. Values: cx={norm_center_x}, cy={norm_center_y}, w={norm_width}, h={norm_height}")
                 continue


            yolo_lines.append(f"{class_index} {norm_center_x:.6f} {norm_center_y:.6f} {norm_width:.6f} {norm_height:.6f}")
            valid_box_found = True 


        if not valid_box_found:
            skipped_no_axles += 1 
            continue


        # Write YOLO label file
        try:
            with open(label_filepath, 'w') as f:
                f.write("\n".join(yolo_lines))
        except Exception as e:
            print(f"Error writing label file {label_filepath}: {e}")
            skipped_write_error += 1
            continue

        # Copy image file
        try:
            dest_img_path = os.path.join(img_output_dir, base_filename)
            shutil.copy(img_path, dest_img_path)
            count += 1
        except Exception as e:
            print(f"Error copying image file {img_path} to {img_output_dir}: {e}")
            skipped_copy_error += 1
            # Clean up label file if image copy failed
            if os.path.exists(label_filepath):
                 try:
                     os.remove(label_filepath)
                     print(f" - Removed corresponding label file {label_filename}")
                 except Exception as rem_e:
                     print(f" - Failed to remove label file {label_filename}: {rem_e}")


    # Report counts after iterating through the list
    print(f" - Processed items: {count}")
    print(f" - Skipped (no valid axles): {skipped_no_axles}")
    print(f" - Skipped (dimension error): {skipped_dimension_error}")
    print(f" - Skipped (label write error): {skipped_write_error}")
    print(f" - Skipped (image copy error): {skipped_copy_error}")
    return count



#create Output Directories
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)
print(f"Ensured YOLO directory structure exists under: {output_base_dir}")

# load the Initial Data
axle_dataset = load_axle_data(DATASET_DIR)

if axle_dataset:
    # split Data
    try:
        train_data, val_data = train_test_split(
            axle_dataset,
            test_size=0.2,
            random_state=42 
        )
        print(f"\nSplit dataset: {len(train_data)} training, {len(val_data)} validation samples.")
    except ValueError as e:
        print(f"\nError during train/test split: {e}")
        print("Cannot proceed with conversion. Ensure 'axle_dataset' was loaded correctly.")
        train_data, val_data = [], [] 


    if train_data:
        print("\nConverting training data to YOLO format...")
        train_count = convert_to_yolo(train_data, train_images_dir, train_labels_dir, class_index=0) # Assuming 'axle' is class 0
        print(f"Finished converting training data.")

    if val_data:
        print("\nConverting validation data to YOLO format...")
        val_count = convert_to_yolo(val_data, val_images_dir, val_labels_dir, class_index=0) # Assuming 'axle' is class 0
        print(f"Finished converting validation data.")

    #  dataset.yaml file 
    abs_train_images_dir = os.path.abspath(train_images_dir)
    abs_val_images_dir = os.path.abspath(val_images_dir)

    # Content for the YAML file
    yaml_content = f"""
train: {abs_train_images_dir}
val: {abs_val_images_dir}

# number of classes
nc: 1

# class names (index 0 corresponds to 'axle')
names: ['axle']
"""

    yaml_filepath = os.path.join(output_base_dir, "dataset.yaml")
    try:
        with open(yaml_filepath, 'w') as f:
            f.write(yaml_content)
        print(f"\nCreated dataset configuration file: {yaml_filepath}")
        print(f"Train path in YAML: {abs_train_images_dir}")
        print(f"Val path in YAML: {abs_val_images_dir}")
    except Exception as e:
        print(f"\nError writing dataset.yaml file: {e}")

else:
    print("\n'axle_dataset' is empty. No data loaded, cannot proceed with split and conversion.")

print("\nScript finished.")

In [ ]:
from ultralytics import YOLO
import os

# Path to the dataset.yaml 
yaml_path = r"D:\sem 5\DL\Project\dataset2\Trucks_YOLO\dataset.yaml"

model_name = 'yolov8n.pt'

results_dir = r"D:\sem 5\DL\Project\dataset2\model"
os.makedirs(results_dir, exist_ok=True) 

EPOCHS = 50

print("--- Starting YOLOv8 Training ---")
print(f"Dataset configuration: {yaml_path}")
print(f"Pre-trained model: {model_name}")
print(f"Number of epochs: {EPOCHS}")
print(f"Results will be saved in: {results_dir}")

model = YOLO(model_name)

# training
results = model.train(
    data=yaml_path,        
    epochs=EPOCHS,         
    imgsz=640,             
    project=results_dir,   
    name='axle_detection'  
)

print("\n--- Training Finished ---")
print(f"Results saved in: {os.path.join(results_dir, 'axle_detection')}")


In [ ]:
from IPython.display import Image

# results.png file 
results_graph_path = r"D:\sem 5\DL\Project\dataset2\model\axle_detection\results.png"

Image(filename=results_graph_path)

In [ ]:
from ultralytics import YOLO

model_path = r"D:\sem 5\DL\Project\dataset2\model\axle_detection\weights\best.pt"

model = YOLO(model_path)

results = model.predict(source=r'D:\sem 5\DL\Project\dataset2\Trucks_YOLO\train\images\20170418075806_color-[ROI-1]-27.jpg', save=True)

print("Prediction finished. Results saved.")


image 1/1 D:\sem 5\DL\Project\dataset2\Trucks_YOLO\train\images\20170418075806_color-[ROI-1]-27.jpg: 288x640 1 axle, 138.7ms
Speed: 5.5ms preprocess, 138.7ms inference, 25.0ms postprocess per image at shape (1, 3, 288, 640)
Results saved to D:\sem 5\DL\Project\dataset2\runs\detect\predict3
Prediction finished. Results saved.


In [ ]:
from ultralytics import YOLO
import cv2 
import os

model_path = r'D:\sem 5\DL\Project\dataset2\model\axle_detection\weights\best.pt'
model = YOLO(model_path)

test_image_path = r'E:\downloads_2\photo_2025-10-27_20-02-13 (2).jpg'
output_dir = r'D:\sem 5\DL\Project\dataset2\runs\detect'
os.makedirs(output_dir, exist_ok=True)

results = model(
    test_image_path,
    conf=0.065,        # low to catch all candidates
    iou=0.2          # Increase IoU threshold to merge closer boxes (default = ~0.7)
)


axle_count = 0
img = cv2.imread(test_image_path)

for result in results:
    boxes = result.boxes  
    for box in boxes:
        # if the detected class index is 0 (axle)
        if int(box.cls[0]) == 0:
            axle_count += 1
            coords = box.xyxy[0].tolist()
            coords = [int(c) for c in coords] 
            x1, y1, x2, y2 = coords

            #  confidence score
            conf = box.conf[0]

            # draw rectangle on the image
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2) # Green box, thickness 2
            # Add label and confidence
            label = f"Axle: {conf:.2f}"
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display
print(f"Detected {axle_count} axles in {os.path.basename(test_image_path)}")

# Save the image with boxes drawn
output_filename = os.path.basename(test_image_path)
output_path = os.path.join(output_dir, output_filename)
cv2.imwrite(output_path, img)
print(f"Result image saved to: {output_path}")


image 1/1 E:\downloads_2\photo_2025-10-27_20-02-13 (2).jpg: 384x640 1 axle, 162.0ms
Speed: 7.0ms preprocess, 162.0ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
Detected 1 axles in photo_2025-10-27_20-02-13 (2).jpg
Result image saved to: D:\sem 5\DL\Project\dataset2\runs\detect\photo_2025-10-27_20-02-13 (2).jpg


In [ ]:
import os
import glob

DATASET_DIR = r"D:\sem 5\DL\Project\dataset2\Trucks"

# List all image files
image_extensions = ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG', '*.png', '*.PNG']
all_images = []
for ext in image_extensions:
    all_images.extend(glob.glob(os.path.join(DATASET_DIR, ext)))

print(f"Found {len(all_images)} image files:")
for img in sorted(all_images)[:2000]:  # Show first 20
    print(img)